In [1]:
import numpy as np
from scipy.sparse import coo_matrix
from elemental_3D import elemental_3D
from numpy.linalg import pinv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ansys.mapdl.core import launch_mapdl
import pandas as pd

nelx = 2
nely = 1
nelz = 1
rho = 0.5
theta = 0.1
phi = 0

#Initialisation 
D = {}
D['nnode'] = (nelx + 1) * (nely + 1) * (nelz + 1)  # number of nodes in model
D['nel'] = nelx * nely * nelz  # number of elements in model
D['etype'] = 3  # element type (only one per model)
D['nelx'] = nelx
D['nely'] = nely
D['nelz'] = nelz
D['elname'] = 'elemental_3D'
D['ndof'] = 3
D['nenode'] = 8

nnode = (nelx + 1) * (nely + 1) * (nelz + 1)  # number of nodes in model
nel= nelx * nely * nelz  # number of elements in model
etype = 3  # element type (only one per model)
ndof = 3 # nombre de degré de liberté pour chaque noeud
nenode = 8 # nombre de noeuds pour chaque element

# Génération du maillage
xloc1, yloc1, zloc1 = np.meshgrid(np.arange(nelx + 1),np.arange(nely, -1, -1),np.arange(nelz + 1),indexing='ij')

""""
# Conversion en vecteurs pour affichage
x = xloc1.flatten()
y = yloc1.flatten()
z = zloc1.flatten()

# Affichage avec Matplotlib
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, color='b', s=10)  # Nuage de points

# Labels des axes
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Visualisation du maillage structuré")

plt.show()
"""

# 🔹 Lancer MAPDL
mapdl = launch_mapdl()

# 🔹 Définition des dimensions
lx, ly, lz = 50, 50, 50  # Longueurs du domaine (modifiable)

# 🔹 Entrer dans le mode PREP7 (préprocesseur)
mapdl.prep7()

# 🔹 Création d'un bloc 3D
mapdl.block(0, nelx, 0, nely, 0, nelz)

# 🔹 Définition des propriétés du matériau
E = 210e9  # Module de Young (Pa) (Acier)
nu = 0.3   # Coefficient de Poisson
mapdl.mp("EX", 1, E)
mapdl.mp("PRXY", 1, nu)

# 🔹 Définition du maillage
mapdl.et(1, "SOLID185")  # Type d'élément 3D (modifiez selon vos besoins)
mapdl.esize(1)   # Taille élémentaire pour homogénéité
mapdl.vmesh(1)  # Maillage du volume
#print(mapdl.mesh.nnum)  # Liste des numéros de nœuds
#print(mapdl.mesh.enum)  # Liste des numéros d’éléments

# Pour obtenir juste les numéros
node_vect = mapdl.mesh.nnum
node_mat = node_vect.reshape((nely + 1, nelx + 1, nelz + 1), order='F')
#print(node_mat)

"""
# Sélectionner tous les nœuds
mapdl.allsel()

# Récupérer le nombre total de nœuds générés après le maillage
num_nodes = len(mapdl.mesh.nnum)

# Afficher le résultat
print(f"Nombre total de nœuds générés : {num_nodes}")
"""

# 🔹 Sélection des nœuds de la face X = 0 (plan YZ)
mapdl.nsel("S", "LOC", "X", 0)

# 🔹 Appliquer un encastrement (tous les degrés de liberté fixés)
mapdl.d("ALL", "UX", 0)
mapdl.d("ALL", "UY", 0)
mapdl.d("ALL", "UZ", 0)

# 🔹 Sélection des nœuds de l'arête supérieure (X = lx, Y = ly, Z = lz)
mapdl.nsel("S", "LOC", "X", nelx)
mapdl.nsel("R", "LOC", "Z", nelz)  # Réduction à Z = lz

# 🔹 Application de la force en Z sur ces nœuds
force_value = -1e6  # Force descendante (N)
mapdl.f("ALL", "FZ", force_value)
mapdl.finish()

# 🔹 Passer en mode solution (SOLU)
mapdl.slashsolu()  # Mode solution
mapdl.solve()      # Exécuter l'analyse statique
mapdl.finish()     # Quitter le mode solution

# 🔹 Mode post-traitement
mapdl.post1()
mapdl.set(1)  # Lire la première solution

# 🔹 Récupérer les déplacements nodaux
mapdl.allsel()
ux = mapdl.post_processing.nodal_displacement("X")
uy = mapdl.post_processing.nodal_displacement("Y")
uz = mapdl.post_processing.nodal_displacement("Z")
# Conversion en tableaux NumPy
ux = np.array(ux)
uy = np.array(uy)
uz = np.array(uz)

# Construction du tableau combiné [[UX, UY, UZ] pour chaque nœud]
disp_vector = np.column_stack((ux, uy, uz))

connectivity = mapdl.mesh.elem
#print(connectivity)
# Nettoyage : extraire uniquement la liste des nœuds par élément
cleaned_connectivity = [[int(node) for node in elem] for elem in connectivity]

# Affichage formaté
ielem = [elem[-8:] for elem in cleaned_connectivity]
#print(ielem)

# 🔹 Affichage du maillage et des conditions de support
#mapdl.eplot()  # Affichage des éléments
#mapdl.eplot(show_node_numbering=True, show_element_numbering=True)

"""
# 🔹 Vérification des conditions appliquées
#print("Nœuds encastrés :")
mapdl.nsel("S", "LOC", "X", 0)
#print(mapdl.nlist())  # Liste des nœuds encastrés

#print("Forces appliquées :")
mapdl.nsel("S", "LOC", "X", lx)
mapdl.nsel("R", "LOC", "Y", ly)
mapdl.nsel("R", "LOC", "Z", lz)
#print(mapdl.flist())  # Liste des forces appliquées

mapdl.allsel()  # Sélectionner tous les nœuds
num_nodes = mapdl.get(entity="NODE", item1="NUM", it1num="MAX")  # Récupérer le nombre total
print(f"Nombre total de nœuds générés : {num_nodes}")
#print(mapdl.nlist())  # Afficher la liste des nœuds
"""

# 🔹 Fermeture de MAPDL (désactiver si vous voulez faire d'autres analyses)
mapdl.exit()

In [5]:
import numpy as np
from scipy.sparse import coo_matrix
from elemental_3D import elemental_3D
from numpy.linalg import pinv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ansys.mapdl.core import launch_mapdl

nelx = 2
nely = 2
nelz = 2
rho = 0.5
theta = 0.1
phi = 0

#Initialisation 
D = {}
D['nnode'] = (nelx + 1) * (nely + 1) * (nelz + 1)  # number of nodes in model
D['nel'] = nelx * nely * nelz  # number of elements in model
D['etype'] = 3  # element type (only one per model)
D['nelx'] = nelx
D['nely'] = nely
D['nelz'] = nelz
D['elname'] = 'elemental_3D'
D['ndof'] = 3
D['nenode'] = 8

nnode = (nelx + 1) * (nely + 1) * (nelz + 1)  # number of nodes in model
nel= nelx * nely * nelz  # number of elements in model
etype = 3  # element type (only one per model)
ndof = 3 # nombre de degré de liberté pour chaque noeud
nenode = 8 # nombre de noeuds pour chaque element

# Génération du maillage
xloc1, yloc1, zloc1 = np.meshgrid(np.arange(nelx + 1),np.arange(nely, -1, -1),np.arange(nelz + 1),indexing='ij')

# 🔹 Lancer MAPDL
mapdl = launch_mapdl()

# 🔹 Entrer dans le mode PREP7 (préprocesseur)
mapdl.prep7()

# 🔹 Création d'un bloc 3D
mapdl.block(0, nelx, 0, nely, 0, nelz)

# 🔹 Définition des propriétés du matériau
E = 210e9  # Module de Young (Pa) (Acier)
nu = 0.3   # Coefficient de Poisson
mapdl.mp("EX", 1, E)
mapdl.mp("PRXY", 1, nu)

# 🔹 Définition du maillage
mapdl.et(1, "SOLID185")  # Type d'élément 3D (modifiez selon vos besoins)
mapdl.esize(1)   # Taille élémentaire pour homogénéité
mapdl.vmesh('ALL')  # Maillage du volume

# 🔹 Sélection des nœuds de la face X = 0 (plan YZ)
mapdl.nsel("S", "LOC", "X", 0)

# 🔹 Appliquer un encastrement (tous les degrés de liberté fixés)
mapdl.d("ALL", "UX", 0)
mapdl.d("ALL", "UY", 0)
mapdl.d("ALL", "UZ", 0)

# 🔹 Sélection des nœuds de l'arête supérieure (X = lx, Y = ly, Z = lz)
mapdl.nsel("S", "LOC", "X", nelx)
mapdl.nsel("R", "LOC", "Z", nelz)  # Réduction à Z = lz

# 🔹 Application de la force en Z sur ces nœuds
force_value = -1e6  # Force descendante (N)
mapdl.f("ALL", "FZ", force_value)
mapdl.finish()

# 🔹 Passer en mode solution (SOLU)
mapdl.slashsolu()  # Mode solution
mapdl.solve()      # Exécuter l'analyse statique

# 🔹 Mode post-traitement
mapdl.post1()
mapdl.set(1)  # Lire la première solution

# 🔹 Récupérer les déplacements nodaux
mapdl.allsel()

# 🔹 Récupérer la matrice de rigidité globale
K_global = mapdl.k()

# Récupération des déplacements nodaux pour chaque direction
ux = mapdl.post_processing.nodal_displacement("X")
uy = mapdl.post_processing.nodal_displacement("Y")
uz = mapdl.post_processing.nodal_displacement("Z")

# Conversion en tableaux NumPy
ux = np.array(ux)
uy = np.array(uy)
uz = np.array(uz)

# Construction du tableau combiné [[UX, UY, UZ] pour chaque nœud]
utot = np.column_stack((ux, uy, uz))

connectivity = mapdl.mesh.elem
#print(connectivity)
# Nettoyage : extraire uniquement la liste des nœuds par élément
cleaned_connectivity = [[int(node) for node in elem] for elem in connectivity]

# Affichage formaté
ielem = [elem[-8:] for elem in cleaned_connectivity]
#print(ielem)

# 🔹 Affichage du maillage et des conditions de support
mapdl.eplot(show_node_numbering=True, show_element_numbering=True)

# 🔹 Fermeture de MAPDL (désactiver si vous voulez faire d'autres analyses)
mapdl.exit()

MapdlInvalidRoutineError: *** WARNING ***                         CP =       0.000   TIME= 00:00:00
 K is not a recognized POST1 command, abbreviation, or macro.            
  

Ignore these messages by setting 'ignore_errors'=True.


In [11]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# 🔹 Lancer MAPDL sur un port spécifique
mapdl = launch_mapdl(port=50059)

# 🔹 Effacer l'environnement précédent et entrer dans PREP7
mapdl.clear()
mapdl.prep7()

# 🔹 Définition des paramètres du maillage
nelx, nely, nelz = 2, 2, 2  # 2 éléments dans chaque direction

# 🔹 Création d'un bloc 3D
mapdl.block(0, nelx, 0, nely, 0, nelz)

# 🔹 Définition des propriétés du matériau
E = 210e9  # Module de Young (Pa)
nu = 0.3   # Coefficient de Poisson
mapdl.mp("EX", 1, E)
mapdl.mp("PRXY", 1, nu)

# 🔹 Définition du maillage
mapdl.et(1, "SOLID185")  # Élément solide 3D
mapdl.esize(1)  # Taille élémentaire
mapdl.vmesh('ALL')  # Maillage du volume

# 🔹 Appliquer les conditions aux limites
mapdl.nsel("S", "LOC", "X", 0)  # Sélectionner les nœuds du plan X=0
mapdl.d("ALL", "UX", 0)
mapdl.d("ALL", "UY", 0)
mapdl.d("ALL", "UZ", 0)
mapdl.allsel()  # Sélectionner tous les nœuds
# Extraire la matrice de rigidité
mapdl.slashsolu()
mapdl.solve()
mapdl.finish()

# Exporter la matrice de rigidité
# Exporter la matrice de rigidité de la sous-structure
mapdl.subopt("EXPORT", "SUB1", "STIFF", "stiffness_matrix.txt")


mapdl.exit()

TypeError: DynamicOptions.subopt() takes from 1 to 3 positional arguments but 5 were given